# All Sites- 0 NetCDF
## taking all the data, and putting into a clear nedcdf or pickle for later (to move to random forest analysis)
### then going to run the simulations on the linux box to save my local machine

In [1]:
# import packages 
%matplotlib widget

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots # adding for subplots
import plotly.figure_factory as ff

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import csv 
import copy 
import os.path 

# Import All Data

## Sodankyla Data

In [2]:
# OPEN MET DATA 
sod =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Sodankyla/SOD_1819_1hr_cleaned.csv")
sod['datetime'] = pd.to_datetime(sod['datetime'])

# OPEN CLASSIFICATIONS
sodobs = pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/datetimeformat_classifications_sodankyla2019_type.csv")
sodobs['datetime'] = pd.to_datetime(sodobs['datetime'])
sodobs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed 

# MERGE
soddf = pd.merge(sod, sodobs, how='outer', on='datetime')
soddf.index = pd.DatetimeIndex(soddf['datetime'])
soddf.drop(columns=['datetime'], inplace=True)

# Create seperate df for only snow in the canopy timesteps 
sod_df_unload = soddf.copy(deep=True)
sod_df_unload = sod_df_unload.dropna(axis=0, subset=['CD'])

# Have to remove all nans for this plotting to work.... need to come back to this 
# sod_df_unload.dropna(inplace=True)

# Create sunlit column
sod_df_unload['Sunlit'] = sod_df_unload.E.copy(deep=True)
sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
sod_df_unload['Classification'] = sod_df_unload.CD.copy(deep=True)
sod_df_unload.Classification.mask(sod_df_unload.Classification == 0, "Snow Unloading", inplace=True)
sod_df_unload.Classification.mask(sod_df_unload.Classification == 1, "Snow Staying", inplace=True)

# sod_df_snowstay   = sod_df_unload.where(sod_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
# sod_df_snowunload = sod_df_unload.where(sod_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

In [3]:
#---------------------------------------------------------------------------------------------------------------------------


# https://plotly.com/python/marker-style
# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
sod_df_unload['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
sod_df_unload['Typesymbol'].mask(sod_df_unload['Type'] == 1, 102, inplace=True) 
sod_df_unload['Typesymbol'].mask(sod_df_unload['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_unload['Typesymbol'] = sod_df_unload['Typesymbol'].fillna(128)  

# # SNOW-------------------------------------------------------
# Create symbols for sunlit or not
sod_df_unload['Snowsymbol'] = np.nan

# open triangle for melt, diamond for sub, open x circle for no classification
sod_df_unload['Snowsymbol'].mask(sod_df_unload['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
sod_df_unload['Snowsymbol'].mask(sod_df_unload['Snow'] == 'sub', 18, inplace=True)  
sod_df_unload['Snowsymbol'].mask(sod_df_unload['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_unload['Snowsymbol'] = sod_df_unload['Snowsymbol'].fillna(128)  


#---------------------------------------------------------------------------------------------------------------------------

## Laret

In [4]:
# Open met dataset
opn  = pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/MetStations/Cleaned/opnc_hcass.csv")
opn['datetime'] = pd.to_datetime(opn['datetime'])

# Open observations classifications 
obs = pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/Excel/Datetime/datetimeformat_ccg_classifications_only_cutdates_type.csv")
obs['datetime'] = pd.to_datetime(obs['datetime'])
obs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed

# Merge them
opndf = pd.merge(opn, obs, how='outer', on='datetime')
opndf.index = pd.DatetimeIndex(opndf['datetime'])

# Create seperate df for only snow in the canopy timesteps 
opn_df_unload = opndf.copy(deep=True)
opn_df_unload = opn_df_unload.dropna(axis=0, subset=['CD'])

# # Have to remove all nans for this plotting to work.... need to come back to this 
# opn_df_unload.dropna(inplace=True)

# SPECIAL FOR LARET BECAUSE OF NAMES 
opn_df_unload['temp'] = opn_df_unload['AirTC_Avg'].copy(deep=True)
opn_df_unload['shortwave'] = opn_df_unload['SWR_Avg'].copy(deep=True)
opn_df_unload['windspeed'] = opn_df_unload['WS_ms_Avg'].copy(deep=True)

# Create sunlit column
opn_df_unload['Sunlit'] = opn_df_unload.E.copy(deep=True)
opn_df_unload.Sunlit.mask(opn_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
opn_df_unload.Sunlit.mask(opn_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
opn_df_unload['Classification'] = opn_df_unload.CD.copy(deep=True)
opn_df_unload.Classification.mask(opn_df_unload.Classification == 0, "Snow Unloading", inplace=True)
opn_df_unload.Classification.mask(opn_df_unload.Classification == 1, "Snow Staying", inplace=True)

# opn_df_snowstay   = opn_df_unload.where(opn_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
# opn_df_snowunload = opn_df_unload.where(opn_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

In [5]:
#---------------------------------------------------------------------------------------------------------------------------

# https://plotly.com/python/marker-style
# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
opn_df_unload['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
opn_df_unload['Typesymbol'].mask(opn_df_unload['Type'] == 1, 102, inplace=True) 
opn_df_unload['Typesymbol'].mask(opn_df_unload['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
opn_df_unload['Typesymbol'] = opn_df_unload['Typesymbol'].fillna(128)  

# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
opn_df_unload['Snowsymbol'] = np.nan

# open triangle for melt, diamond for sub, open x circle for no classification
opn_df_unload['Snowsymbol'].mask(opn_df_unload['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
opn_df_unload['Snowsymbol'].mask(opn_df_unload['Snow'] == 'sub', 18, inplace=True)  
opn_df_unload['Snowsymbol'].mask(opn_df_unload['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
opn_df_unload['Snowsymbol'] = opn_df_unload['Snowsymbol'].fillna(128)  


#---------------------------------------------------------------------------------------------------------------------------

## Niwot

In [6]:
# AMERIFLUX MET DATA 
niwotflux =   pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/niwot_2017_ameriflux_unload1hr.csv")
niwotflux['datetime']  = pd.DatetimeIndex(niwotflux['datetime'])

# OBSERVATIONS CLASSIFICATIONS
niwotobs =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/datetimeformat_classifications_niwot2017_type.csv")
niwotobs['datetime'] = pd.to_datetime(niwotobs['datetime'])

niwotobs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed 

# MERG THEM 
niwotdf = pd.merge(niwotflux, niwotobs, how='outer', on='datetime')
niwotdf.index = pd.DatetimeIndex(niwotdf['datetime'])

# Create seperate df for only snow in the canopy timesteps 
niwot_df_unload = niwotdf.copy(deep=True)
niwot_df_unload = niwot_df_unload.dropna(axis=0, subset=['CD'])

# # Have to remove all nans for this plotting to work.... need to come back to this 
# niwot_df_unload.dropna(inplace=True)

# Create sunlit column
niwot_df_unload['Sunlit'] = niwot_df_unload.E.copy(deep=True)
niwot_df_unload.Sunlit.mask(niwot_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
niwot_df_unload.Sunlit.mask(niwot_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
niwot_df_unload['Classification'] = niwot_df_unload.CD.copy(deep=True)
niwot_df_unload.Classification.mask(niwot_df_unload.Classification == 0, "Snow Unloading", inplace=True)
niwot_df_unload.Classification.mask(niwot_df_unload.Classification == 1, "Snow Staying", inplace=True)

# niwot_df_snowstay   = niwot_df_unload.where(niwot_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
# niwot_df_snowunload = niwot_df_unload.where(niwot_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

In [7]:
#---------------------------------------------------------------------------------------------------------------------------


# https://plotly.com/python/marker-style
# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
niwot_df_unload['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
niwot_df_unload['Typesymbol'].mask(niwot_df_unload['Type'] == 1, 102, inplace=True) 
niwot_df_unload['Typesymbol'].mask(niwot_df_unload['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwot_df_unload['Typesymbol'] = niwot_df_unload['Typesymbol'].fillna(128)  

# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
niwot_df_unload['Snowsymbol'] = np.nan

# open triangle for melt, diamond for sub, open x circle for no classification
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
niwot_df_unload['Snowsymbol'].mask(niwot_df_unload['Snow'] == 'sub', 18, inplace=True)  
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwot_df_unload['Snowsymbol'] = niwot_df_unload['Snowsymbol'].fillna(128)  


#---------------------------------------------------------------------------------------------------------------------------

### Plotting Constants

In [8]:
## Define some plotting constants for easier coding 
plt.close('all')

# Colors
colornosnow = '#D2B48C' # nice tan
colorsnow = '#7dcfd4' # slightly desaturated cyan
colorsnowunload = '#1F15D5' # bright, deep blue 
colorsunny = '#E4E44A' # trying a little less bright 
unloadcolors = [colorsnow, colorsnowunload]
colorscaled = [[0., colorsnowunload], [1, colorsnow]]

# Names
namesnow = 'snow staying'
namesnowunload = 'snow Unloading'
group_labels = ['snow staying', 'snow snloading']

# namelaret = 'Davos Laret, Switzerland'
# nameniwot = 'Niwot Ridge, Colorado, USA'
# namehopper = 'Mount Hopper, Washington, USA'
# namesodankyla = 'Sodankylä, Finland'

namesodankyla = 'A)  Sodankylä, Finland'
namesod = 'A)  Sodankyla, Finland'
namelaret = 'B)  Davos Laret, Switzerland'
nameniwot = 'C)  Niwot Ridge, Colorado, USA'

nametemperature = "air temperature (℃)"
namewindspeed = "wind speed (ms⁻¹)"
nameshortwave = "shortwave radiation (Wm⁻²)"

print(nametemperature)
print(namewindspeed)
print(nameshortwave)

## Putting all the data together for plots

In [9]:
niwot_df_unload.head()

,datetime,shortwave,longwave,precip,windspeed,RH,temp,A,B,CD,E,Type,Class,Snow,Sunlit,Classification,Typesymbol,Snowsymbol
datetime,,,,,,,,,,,,,,,,,,
2016-10-06 06:00:00,2016-10-06 06:00:00,4.93,297.470,0.000,1.180,96.650,-3.495,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 07:00:00,2016-10-06 07:00:00,29.43,298.655,0.000,1.735,93.195,-3.325,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 08:00:00,2016-10-06 08:00:00,124.50,309.965,0.225,2.080,83.190,-2.430,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 09:00:00,2016-10-06 09:00:00,102.04,306.465,0.700,2.745,90.135,-3.250,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 10:00:00,2016-10-06 10:00:00,614.38,240.145,0.075,4.595,80.390,-3.350,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0


In [10]:
niwot_df_unload.drop(columns=['precip'], inplace=True)

In [11]:
niwot_df_unload.head()

,datetime,shortwave,longwave,windspeed,RH,temp,A,B,CD,E,Type,Class,Snow,Sunlit,Classification,Typesymbol,Snowsymbol
datetime,,,,,,,,,,,,,,,,,
2016-10-06 06:00:00,2016-10-06 06:00:00,4.93,297.470,1.180,96.650,-3.495,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 07:00:00,2016-10-06 07:00:00,29.43,298.655,1.735,93.195,-3.325,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 08:00:00,2016-10-06 08:00:00,124.50,309.965,2.080,83.190,-2.430,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 09:00:00,2016-10-06 09:00:00,102.04,306.465,2.745,90.135,-3.250,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2016-10-06 10:00:00,2016-10-06 10:00:00,614.38,240.145,4.595,80.390,-3.350,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0


In [12]:
sod_df_unload.head()

,shortwave,longwave,ppt_1,ppt1,RH,windspeed,pressure,snowdepth,temp,A,B,CD,E,Type,Class,Snow,Sunlit,Classification,Typesymbol,Snowsymbol
datetime,,,,,,,,,,,,,,,,,,,,
2018-10-05 14:00:00,36.25,312.55,0.000379,1.630000e-12,98.0,3.61035,96730.0,7.0,0.15,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 15:00:00,39.60,312.30,0.000217,1.300000e-12,97.5,2.38995,96635.0,8.0,0.20,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 16:00:00,23.25,312.05,0.000126,5.450000e-13,97.0,1.27125,96595.0,7.0,0.15,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 17:00:00,2.10,311.30,0.000054,1.200000e-13,97.5,0.76275,96610.0,9.0,0.05,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 18:00:00,0.00,309.70,0.000072,8.200000e-14,98.0,1.27125,96650.0,9.0,-0.05,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0


In [13]:
sod_df_unload.drop(columns=['ppt1', 'ppt_1'], inplace=True)

In [14]:
sod_df_unload.head()

,shortwave,longwave,RH,windspeed,pressure,snowdepth,temp,A,B,CD,E,Type,Class,Snow,Sunlit,Classification,Typesymbol,Snowsymbol
datetime,,,,,,,,,,,,,,,,,,
2018-10-05 14:00:00,36.25,312.55,98.0,3.61035,96730.0,7.0,0.15,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 15:00:00,39.60,312.30,97.5,2.38995,96635.0,8.0,0.20,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 16:00:00,23.25,312.05,97.0,1.27125,96595.0,7.0,0.15,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 17:00:00,2.10,311.30,97.5,0.76275,96610.0,9.0,0.05,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0
2018-10-05 18:00:00,0.00,309.70,98.0,1.27125,96650.0,9.0,-0.05,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0


In [15]:
opn_df_unload.head()

,datetime,SWR_Avg,LWRin_Avg,AirTC_Avg,RH,IRTemp_C_A,AirTC2_Avg,WindDir,WS_ms_Avg,A,...,Type,Class,Snow,temp,shortwave,windspeed,Sunlit,Classification,Typesymbol,Snowsymbol
datetime,,,,,,,,,,,,,,,,,,,,,
2021-02-13 12:00:00,2021-02-13 12:00:00,604.682917,188.964583,-8.376617,71.224333,-6.834288,-8.540871,49.450000,2.645875,1.0,...,1.0,sw,melt,-8.376617,604.682917,2.645875,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 13:00:00,2021-02-13 13:00:00,554.419583,190.445417,-7.641083,70.487708,-6.324933,-7.850617,53.912500,2.669346,1.0,...,1.0,sw,melt,-7.641083,554.419583,2.669346,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 14:00:00,2021-02-13 14:00:00,468.039167,191.897083,-8.342483,77.139792,-7.237567,-8.369750,41.700000,3.364821,1.0,...,1.0,sw,melt,-8.342483,468.039167,3.364821,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 15:00:00,2021-02-13 15:00:00,356.765833,202.591667,-8.621583,79.103375,-8.070804,-8.626625,52.441667,2.694854,1.0,...,1.0,sw,melt,-8.621583,356.765833,2.694854,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 16:00:00,2021-02-13 16:00:00,79.754583,187.632500,-10.212458,85.675458,-15.041375,-10.153000,73.564854,1.706025,1.0,...,NaN,NaN,NaN,-10.212458,79.754583,1.706025,Not Sunlit,Snow Staying,128.0,128.0


In [16]:
opn_df_unload.drop(columns=['SWR_Avg', 'LWRin_Avg', 'AirTC_Avg', 'AirTC2_Avg' ,'IRTemp_C_A', 'WindDir', 'WS_ms_Avg' ], inplace=True)

In [17]:
opn_df_unload.head()

,datetime,RH,A,B,CD,E,Type,Class,Snow,temp,shortwave,windspeed,Sunlit,Classification,Typesymbol,Snowsymbol
datetime,,,,,,,,,,,,,,,,
2021-02-13 12:00:00,2021-02-13 12:00:00,71.224333,1.0,0.0,0.0,1.0,1.0,sw,melt,-8.376617,604.682917,2.645875,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 13:00:00,2021-02-13 13:00:00,70.487708,1.0,0.0,0.0,1.0,1.0,sw,melt,-7.641083,554.419583,2.669346,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 14:00:00,2021-02-13 14:00:00,77.139792,1.0,0.0,0.0,1.0,1.0,sw,melt,-8.342483,468.039167,3.364821,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 15:00:00,2021-02-13 15:00:00,79.103375,1.0,0.0,0.0,1.0,1.0,sw,melt,-8.621583,356.765833,2.694854,Sunlit Canopy,Snow Unloading,102.0,2.0
2021-02-13 16:00:00,2021-02-13 16:00:00,85.675458,1.0,0.0,1.0,0.0,NaN,NaN,NaN,-10.212458,79.754583,1.706025,Not Sunlit,Snow Staying,128.0,128.0


In [18]:
niwot_df_unload['site'] = 'niwot'
sod_df_unload['site'] = 'sod'
opn_df_unload['site'] = 'laret'


In [19]:
all_unload = pd.concat([niwot_df_unload, sod_df_unload, opn_df_unload])

In [20]:
type(all_unload)

pandas.core.frame.DataFrame

# Save to a tidy DF

In [21]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is C278-9FFE

 Directory of c:\Users\Lumbr\OneDrive - UW\Documents\Washington\UnloadingRegimes\Python

09/12/2023  04:58 PM    <DIR>          .
09/12/2023  04:58 PM    <DIR>          ..
02/07/2023  09:32 AM    <DIR>          .ipynb_checkpoints
09/12/2023  04:58 PM           274,918 AllSites_Analysis_0_netcdf.ipynb
08/23/2023  05:39 PM           871,549 AllSites_Analysis_1_box.ipynb
08/24/2023  11:52 AM         1,082,416 AllSites_Analysis_2_scatter.ipynb
09/12/2023  12:22 PM         3,393,415 AllSites_Analysis_2_scatterb.ipynb
08/30/2023  02:41 PM           550,736 AllSites_Analysis_3_duration.ipynb
08/23/2023  05:34 PM         1,409,523 AllSites_Analysis_4_hist.ipynb
08/23/2023  03:05 PM           516,455 AllSites_Analysis_5_pdfs.ipynb
09/16/2022  02:23 AM            17,833 AllSites_CreateEvents.ipynb
09/12/2023  12:42 PM           218,698 allsites_snowunload.csv
06/23/2022  02:21 AM    <DIR>          Environment_Files
07/28/2022  

In [23]:
# path = 'c:\Users\Lumbr\OneDrive - UW\Documents\Washington\UnloadingRegimes\Python'
all_unload.to_csv('allsites_snowunload.csv', index=False)

In [24]:
testing_csv = pd.read_csv("allsites_snowunload.csv")
testing_csv

,datetime,shortwave,longwave,windspeed,RH,temp,A,B,CD,E,Type,Class,Snow,Sunlit,Classification,Typesymbol,Snowsymbol,site,pressure,snowdepth
0,2016-10-06 06:00:00,4.930000,297.470,1.180000,96.650000,-3.495000,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0,niwot,NaN,NaN
1,2016-10-06 07:00:00,29.430000,298.655,1.735000,93.195000,-3.325000,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0,niwot,NaN,NaN
2,2016-10-06 08:00:00,124.500000,309.965,2.080000,83.190000,-2.430000,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0,niwot,NaN,NaN
3,2016-10-06 09:00:00,102.040000,306.465,2.745000,90.135000,-3.250000,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0,niwot,NaN,NaN
4,2016-10-06 10:00:00,614.380000,240.145,4.595000,80.390000,-3.350000,1.0,1.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0,niwot,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,2021-05-03 07:00:00,400.306250,NaN,0.770125,73.653583,2.143021,1.0,0.0,1.0,0.0,NaN,NaN,NaN,Not Sunlit,Snow Staying,128.0,128.0,laret,NaN,NaN
1730,2021-05-03 08:00:00,605.846250,NaN,0.973554,63.681542,3.865325,1.0,0.0,1.0,1.0,NaN,NaN,NaN,Sunlit Canopy,Snow Staying,128.0,128.0,laret,NaN,NaN
1731,2021-05-03 09:00:00,854.292500,NaN,1.648042,57.207667,4.049008,1.0,0.0,0.0,1.0,2.0,tempsw,melt,Sunlit Canopy,Snow Unloading,2.0,2.0,laret,NaN,NaN
1732,2021-05-03 10:00:00,902.823333,NaN,1.427383,48.749292,4.768171,1.0,0.0,0.0,1.0,2.0,tempsw,melt,Sunlit Canopy,Snow Unloading,2.0,2.0,laret,NaN,NaN


## Plot all the sites together

In [25]:
# Class-------------------------------------------------------
# Create symbols for sunlit or not
# State which site we are working with 
df = testing_csv.copy(deep=True)

df['Classsymbol'] = np.nan


# then fillna with 128 for circle with x through it 
df['Classsymbol'] = df['Classsymbol'].fillna(128)  # empty circle


# Some constatns 
colorscaled = [[0., colorsnowunload], [1, colorsnow]]
index_vals = df['CD']
index_symbols = df['Classsymbol']

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[dict(label=nametemp, values=df['temp']),
                                   dict(label=namewind, values=df['windspeed']),
                                   dict(label=nameSW,   values=df['shortwave'])],
                                   marker=dict(color=index_vals, colorscale=colorscaled, symbol=index_symbols, showscale=False, line_color='black', line_width=0.5, size=7),
                                   showlegend=False, showupperhalf=False, diagonal_visible=False,
                      ))

fig.update_layout(title='All Sites Data Together', width=800, height=700, dragmode='select')

fig.show()